***hello Zindi 
in this compitition i tried to use a new Pretrained Model named "ProtBert-BFD model" so thanx to the bioinformatics deep learning
 and specially to Rostlab, because they did the real and the hard part of this solution , ProtBert-BFD is based on Bert model which pretrained on a large corpus of protein sequences in a self-supervised fashion, ***

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
import transformers
from sklearn import preprocessing
from transformers import TFAutoModel, AutoTokenizer, BertTokenizer, TFBertModel
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

print('Using Tensorflow version:', tf.__version__)

In [ ]:
# the tokenizer of prot_bert_bfd
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd")

In [ ]:
train_df = pd.read_csv("../input/instadeep-enzyme-classification-challenge/Train (4).csv")

In [ ]:
test= pd.read_csv("../input/instadeep-enzyme-classification-challenge/Test (3).csv")

In [ ]:
# Encode the sequences

In [ ]:
def bert_encoder(review):
  txt =" ".join(review)
  encoded = tokenizer.encode_plus(txt, add_special_tokens=True, 
                                    max_length=380, 
                                    pad_to_max_length=True, 
                                    return_attention_mask=True, 
                                    return_token_type_ids=True)
  return encoded['input_ids']

Build The Model "

In [ ]:
def build_model(transformer, max_len=380):
    
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    linear= Dense(1024, activation= 'relu')(cls_token)
    drop= Dropout(0.5)(linear)
    out = Dense(20, activation='softmax')(drop) 
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

encode the 20 labels 


In [ ]:
le = preprocessing.LabelEncoder()
le.fit(train_df["LABEL"].unique())
train_df["LABEL"]= le.transform(train_df["LABEL"])

In [ ]:
#encode Train data
bert_train = [bert_encoder(r) for r in tqdm(train_df["SEQUENCE"])]
bert_train = np.array(bert_train)

bert_lbl = [l for l in tqdm(train_df["LABEL"])]
bert_lbl = tf.keras.utils.to_categorical(bert_lbl, num_classes=20)

In [ ]:
#encode Test 
bert_test= [bert_encoder(r) for r in tqdm(test["SEQUENCE"])]
bert_test = np.array(bert_test)

Split our data into Train and validation

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(bert_train, 
                                         bert_lbl,
                                         test_size=0.15, 
                                         random_state=42,
                                        )
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

we will use the Kaggle TPU 

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 1
BATCH_SIZE = 22 * strategy.num_replicas_in_sync
MODEL = "Rostlab/prot_bert_bfd" # bert-base-multilingual-uncased

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_val, y_val))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)



In [ ]:
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(bert_test)
    .batch(BATCH_SIZE)
)

In [ ]:
%%time
with strategy.scope():
    transformer_layer = TFBertModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=380)
model.summary()

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS)

In [ ]:
pred = model.predict(test_dataset, verbose=1)

In [ ]:
pred

In [ ]:
test.head()

In [ ]:
test["LABEL"]= 0

In [ ]:
np.save('kk', pred)

In [ ]:
pred_sentiment = np.argmax(pred, axis=1)

In [ ]:
test["LABEL"]= pred_sentiment

In [ ]:
test["LABEL"]= le.inverse_transform(test["LABEL"])

In [ ]:
cc= ["SEQUENCE_ID", "LABEL"]

In [ ]:
test= test[cc]

In [ ]:
test.to_csv("submission49.csv", index=False)